In [5]:
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import glob
import os
import cv2

1. Import Cubism Images
2. Convert them to grayscale
3. Create array of Cubism labels (represented by 1)


In [6]:
img_dir = "F:\\datamining\\rawdata\\Cubism" 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
cubism_total = []
for f1 in files:
    img = cv2.imread(f1)
    cubism_total.append(img)
cubism_gray = [ color.rgb2gray(i) for i in cubism_total]
cubism_labels = np.ones(len(cubism_gray),dtype=int)

4. Extract HOG features from the grayscale

In [7]:
ppc = 16
hog_images_cubism = []
hog_features_cubism = []
for image in cubism_gray:
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
    hog_images_cubism.append(hog_image)
    hog_features_cubism.append(fd)

Repeat above steps for Romanticism images

In [ ]:
img_dir = "F:\\datamining\\rawdata\\Romanticism" 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
romanticism_total = []
for f1 in files:
    img = cv2.imread(f1)
    romanticism_total.append(img)
romanticism_gray = [ color.rgb2gray(i) for i in romanticism_total]
ppc = 16
hog_images_romanticism = []
hog_features_romanticism = []
for image in romanticism_gray:
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
    hog_images_romanticism.append(hog_image)
    hog_features_romanticism.append(fd)
romanticism_labels=np.zeros(len(romanticism_gray),dtype=int)

Combine both feature sets to get data_total
Combine both label sets to get newlabels

In [11]:
data_total=hog_features_cubism+hog_features_romanticism
cubism_labels.reshape(-1,1)
romanticism_labels.reshape(-1,1)
labels=np.append(cubism_labels,romanticism_labels)
newlabels=np.array(labels).reshape(len(labels),1)
data_total = np.array(data_total)


80% of dataset used for training

In [12]:
percentage = 80
partition = int(len(data_total)*percentage/100)

Trying an SVM Classifier

In [14]:
clf = svm.SVC()
data_total = np.array(data_total)
data_frame = np.hstack((data_total,newlabels))
np.random.shuffle(data_frame)

In [15]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()

In [16]:
clf.fit(x_train,y_train)
y_pred = clf.predict(x_train[51].reshape(1,-1))


Trying a Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))